<img src="../assets/sorting.gif" align="center" width="40%" margin="0" ></img> 

# CODE : COMPANY NAME MATCHER
One of the challenges with querying company names from different databases is the discrepancy in the way entity names are spelt. For example, company “ABC” is shown as “ABC” in the Bloomberg system, but “AB C” in another data source. We have built a python program to make sure they actually refer to the same company.


In [163]:
import requests
import time
import pandas as pd
import numpy as np
import json
from random import randint
import matplotlib.pyplot as plt
import re

from fuzzywuzzy import process
import pycontractions
from cleanco import cleanco



In [158]:
# read in sample company list
companies = pd.read_csv("../data/name_match.csv")
# Convert cols to lowercase 
companies.columns = map(str.lower, companies.columns)
# Check length of df
print("number of company names to check", companies.shape[0])
# Preview first few rows
companies.head()


number of company names to check 23


,system,industry_code,company_name,manual_category
0,NAICS,443142,3 MOBILE TELECOM PTE. LTD.,3 MOBILE TELECOM PTE. LTD.
1,NAICS,453998,3 MOBILE TELECOM PTE. LTD.,3 MOBILE TELECOM PTE. LTD.
2,NAICS,424210,A. MENARINI ASIA-PACIFIC PTE. LTD.,A. MENARINI ASIA-PACIFIC PTE. LTD.
3,NAICS,424210,A. MENARINI ASIA-PACIFIC HOLDINGS PTE. LTD.,A. MENARINI ASIA-PACIFIC PTE. LTD.
4,NAICS,424210,A. MENARINI SINGAPORE PTE. LTD.,A. MENARINI SINGAPORE PTE. LTD.


In [162]:
# Checking for nulls
companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   system         23 non-null     object
 1   industry_code  23 non-null     int64 
 2   company_name   23 non-null     object
dtypes: int64(1), object(2)
memory usage: 680.0+ bytes


In [159]:
# Drop manual_category (guide answers)
companies.drop(columns=['manual_category'], inplace=True)

### READY FOR MATCHING
We have a relatively clean dataset with no nulls. We are ready to move on to Stage 1 of the matching process. 

# STAGE 1: ACCURATE REGISTERED BUSINESS ENTITY NAMES
Let us access the Accounting and Corporate Regulatory Authority's(ACRA) API to cross-check the names in our dataset.

In [147]:
# Defining a function to pre-process text for ACRA API search
# The API's quirk is it does not search strings in brackets well
def acra_process(search_term):
    search_term = search_term.upper() # Change to uppercase
    search_term = re.sub("[\(\[].*?[\)\]]", "", search_term) # Remove text in brackets
    search_term = re.sub("(?:\s)(PTE+\.*)|(?:\s)(LTD+\.*)", "", search_term) # Remove pte. and ltd with or without periods
    search_term = search_term.strip() # Remove spaces at the start and end of the string
    return search_term

In [148]:
# Testing the function
search_term = "  SIMPTE. KOOLTD HEWLETT-PACKARD SINGAPORE (SALES) [TESTING Div] (ASIA) PTE. LTD.   "
acra_process(search_term)

'SIMPTE. KOOLTD HEWLETT-PACKARD SINGAPORE'

In [149]:
# Creating an empty list to nest accurate names
revised_namelist = []
match_accuracy = []
# Defining a function to query ACRA's API for the accurate version of a company's name
def acra_name_matcher(name_series, companies_df):
    print("<<FINDING ACCURATE SPELLING OF COMPANY NAMES IN ACRA>>\n", "-"*52)
    # Create a counter to account for poor/no matches
    no_record_count = 0
    # Looping through all company names
    for search_term in companies_df[name_series]:
        # Reformatting text before querying API
        cleaned_term = acra_process(search_term)
        # Using data.gov's ACRA API 
        query_string='https://data.gov.sg/api/action/datastore_search?resource_id=bdb377b8-096f-4f86-9c4f-85bad80ef93c&q=' + cleaned_term 
        resp = requests.get(query_string)
        #Convert JSON into Python Object 
        data = json.loads(resp.content)
        # Accounting for failures in search
        if data["result"]["records"]==[]:
            # CASE 1 -- NO MATCH: for records that cannot be found in ACRA
            revised_namelist.append("No Match")
            print("Zero Matches Found!")
            print("Search Term: ", search_term)
            # Adding one to no-record counter
            no_record_count += 1
            # Scoring Zero for accuracy
            match_accuracy.append(0)
        else:
            # Create a Dataframe from search results
            search_results = pd.DataFrame(data["result"]["records"])
            # Creating a list of matches
            list_of_matches = search_results["entity_name"][search_results["entity_name"].str.contains(cleaned_term)].tolist()
            
            if search_term in list_of_matches:
                # CASE 2 -- PERFECT MATCH: Appending official company name filed with ACRA
                revised_namelist.append(search_term)
                # Scoring 100 for perfect accuracy
                match_accuracy.append(100)
            else:
                # Adding one to no record counter
                no_record_count += 1
                # CASE 3 -- CLOSE MATCH: for records that are close but not perfect
                revised_namelist.append("Close Match")
                print("Close Matches Found!")
                print("Search Term: ", search_term)
                print("Closest Matches: ", list_of_matches, "\n")
                # Scoring Zero for accuracy
                match_accuracy.append(0)
    # PRINT SUMMARY            
    print("\n---------\n","PERFECT MATCHES: ", len(revised_namelist)-no_record_count , "/", companies_df.shape[0])
    print(" NO MATCH or POOR MATCH: ", no_record_count , "/", companies_df.shape[0])
    # CREATE NEW COLUMNS IN DATAFRAME
    companies_df["official_name"]=revised_namelist
    companies_df["match_accuracy"]=match_accuracy
    return companies_df

In [150]:
# Calling the function on our data
acra_name_matcher("company_name", companies)

<<FINDING ACCURATE SPELLING OF COMPANY NAMES IN ACRA>>
 ----------------------------------------------------
Close Matches Found!
Search Term:  DELFI LTD.
Closest Matches:  ['DELFI LIMITED', 'DELFI PRINTING SERVICES', 'DELFI   F & B LIMITED LIABILITY PARTNERSHIP', 'DELFI CAFETERIA', 'DELFI COMMUNICATIONS ASIA PTE. LTD.', 'DELFI REALTY', 'DELFI YURAKU PTE. LTD.', 'DELFI LEVEL 5 HAIR & BEAUTY SALON', 'DELFI TRADING', 'DELFI SHOES', 'DELFI SINGAPORE PTE. LTD.', 'DELFI COMPUTERS', 'DELFI STUDIOS', 'DELFI GIFTS HOUSE', 'DELFI ONE INVESTMENTS PTE. LTD.', 'DELFI MANAGEMENT SERVICES PTE. LTD.', 'DELFI ENGINEERING CONSTRUCTION', 'DELFI MANAGEMENT SERVICES', 'DELFI REALTY PTE. LTD.', 'DELFI TWO INVESTMENTS PTE. LTD.', 'DELFI THREE INVESTMENTS PTE. LTD.', 'DELFI FLORIST', 'DELFI HAIR STUDIO PTE LTD', 'DELFI CUSTOM TAILOR', 'DELFI COMPUTERS PTE LTD', 'DELFI-ORION PTE. LTD.', 'DELFI UTILITIES SERVICE', 'DELFI MARKETING', 'DELFI STUDIOS PTE LTD', 'JIN DELFI ACCESSORIES PTE LTD'] 

Close Matches Foun

,system,industry_code,company_name,official_name,match_accuracy
0,NAICS,443142,3 MOBILE TELECOM PTE. LTD.,3 MOBILE TELECOM PTE. LTD.,100
1,NAICS,453998,3 MOBILE TELECOM PTE. LTD.,3 MOBILE TELECOM PTE. LTD.,100
2,NAICS,424210,A. MENARINI ASIA-PACIFIC PTE. LTD.,A. MENARINI ASIA-PACIFIC PTE. LTD.,100
3,NAICS,424210,A. MENARINI ASIA-PACIFIC HOLDINGS PTE. LTD.,A. MENARINI ASIA-PACIFIC HOLDINGS PTE. LTD.,100
4,NAICS,424210,A. MENARINI SINGAPORE PTE. LTD.,A. MENARINI SINGAPORE PTE. LTD.,100
5,NAICS,311225,ADEKA (SINGAPORE) PTE. LTD.,ADEKA (SINGAPORE) PTE. LTD.,100
6,NAICS,423910,ADIDAS SINGAPORE PTE LTD,ADIDAS SINGAPORE PTE LTD,100
7,NAICS,424910,ADM ASIA-PACIFIC TRADING PTE. LTD.,ADM ASIA-PACIFIC TRADING PTE. LTD.,100
8,NAICS,424990,ADM ASIA-PACIFIC TRADING PTE. LTD.,ADM ASIA-PACIFIC TRADING PTE. LTD.,100
9,NAICS,423610,ARROW ELECTRONICS ASIA (S) PTE. LTD.,ARROW ELECTRONICS ASIA (S) PTE. LTD.,100


### REVIEWING STAGE 1
> We got quite lucky with stage 1, only four companies do not have a perfect match on ACRA. 

# STAGE 2: INTERNAL MATCHING
For non-perfect matches, we will then create a function to check similarity scores within the data set. 

In [151]:
# Defining a function to pre-process text for an internal matching exercise
def internal_process(search_term):
    search_term = search_term.upper() # Change to uppercase
    search_term = re.sub("[\(\[].*?[\)\]]", "", search_term) # Remove text in brackets
    search_term = re.sub("(?:\s)(PTE+\.*)|(?:\s)(LTD+\.*)", "", search_term) # Remove pte. and ltd with or without periods
    search_term = search_term.strip() # Remove spaces at the start and end of the string
    search_term = cleanco(search_term).clean_name() # Use cleanco's library to remove unexpected legal suffixes like LLP
    return search_term

In [152]:
# Creating an empty list to nest matched values
official_name_list = []
# Defining a function to check for similar company names within the dataset
def comparative_name_match(name_series, official_name_series, accuracy_series, companies_df):
    print("<<MATCHING SIMILAR COMPANIES WITHIN DATASET>>\n", "-"*42)
    # Looping through dataset to isolate currently unmatched companies
    for name in range(companies_df.shape[0]):
        matches = ["No Match" ,"Close Match"]
        if companies_df[official_name_series].iloc[name] in matches:
            # Calling pre-processing function on company name
            string_to_match = internal_process(companies_df[name_series].iloc[name])
            # Calling pre-processing function on perfectly-matched ACRA names through list comprehension
            # Enumerating so that index is present for use later
            string_tuples = [(index, internal_process(word)) for index, word in enumerate(companies_df[official_name_series])]
            string_options = [tup[1] for tup in string_tuples]
            # Using fuzzywuzzy to compute a similarity score between string and ACRA names
            Ratios = process.extract(string_to_match,string_options)
            
            # Print Summary Report
            print("<<Finding Match for: ", string_to_match, " >>")
            print("Top 3 Matches(+ Levenshtein Score): ", Ratios[:3])
            # Extract string with the highest matching percentage
            highest = process.extractOne(string_to_match,string_options)
            print("Top Match: ", highest[0])
            print("Score: ", highest[1])
            # We will accept matches only if scores are 90 and above
            if highest[1] >= 90:
                index = [i for i, string in string_tuples if string == highest[0]]
                new_name = companies_df[official_name_series].iloc[index[0]]
                # Appending new name directly into dataframe
                companies_df.at[name,official_name_series] = new_name
                # Giving the match an accuracy score
                # Minus 10 off accuracy as this is a stage 2 match
                companies_df.at[name,accuracy_series] = highest[1] - 10 
                print("Match Appended!", "\n")
            else:
                print("**LOW SCORE WARNING** -- As the Levenshtein Score is below 90, we will not be providing a match for this company.", "\n")           
    return companies_df
        

In [153]:
# Calling the function on our data
comparative_name_match("company_name", "official_name","match_accuracy", companies)

<<MATCHING SIMILAR COMPANIES WITHIN DATASET>>
 ------------------------------------------
<<Finding Match for:  DELFI  >>
Top 3 Matches(+ Levenshtein Score):  [('DELFI', 100), ('ADEKA', 40), ('3 MOBILE TELECOM', 36)]
Top Match:  DELFI
Levenshtein Score:  100
Match Appended! 

<<Finding Match for:  FOOD EMPIRE HOLDINGS  >>
Top 3 Matches(+ Levenshtein Score):  [('FOOD EMPIRE HOLDINGS', 100), ('A. MENARINI ASIA-PACIFIC HOLDINGS', 86), ('HANWELL HOLDINGS', 64)]
Top Match:  FOOD EMPIRE HOLDINGS
Levenshtein Score:  100
Match Appended! 

<<Finding Match for:  HANWELL HOLDINGS  >>
Top 3 Matches(+ Levenshtein Score):  [('HANWELL HOLDINGS', 100), ('A. MENARINI ASIA-PACIFIC HOLDINGS', 86), ('FOOD EMPIRE HOLDINGS', 64)]
Top Match:  HANWELL HOLDINGS
Levenshtein Score:  100
Match Appended! 

<<Finding Match for:  HOSEN GROUP  >>
Top 3 Matches(+ Levenshtein Score):  [('HOSEN GROUP', 100), ('ARROW ELECTRONICS ASIA', 38), ('FOOD EMPIRE HOLDINGS', 38)]
Top Match:  HOSEN GROUP
Levenshtein Score:  100
Mat

,system,industry_code,company_name,official_name,match_accuracy
0,NAICS,443142,3 MOBILE TELECOM PTE. LTD.,3 MOBILE TELECOM PTE. LTD.,100
1,NAICS,453998,3 MOBILE TELECOM PTE. LTD.,3 MOBILE TELECOM PTE. LTD.,100
2,NAICS,424210,A. MENARINI ASIA-PACIFIC PTE. LTD.,A. MENARINI ASIA-PACIFIC PTE. LTD.,100
3,NAICS,424210,A. MENARINI ASIA-PACIFIC HOLDINGS PTE. LTD.,A. MENARINI ASIA-PACIFIC HOLDINGS PTE. LTD.,100
4,NAICS,424210,A. MENARINI SINGAPORE PTE. LTD.,A. MENARINI SINGAPORE PTE. LTD.,100
5,NAICS,311225,ADEKA (SINGAPORE) PTE. LTD.,ADEKA (SINGAPORE) PTE. LTD.,100
6,NAICS,423910,ADIDAS SINGAPORE PTE LTD,ADIDAS SINGAPORE PTE LTD,100
7,NAICS,424910,ADM ASIA-PACIFIC TRADING PTE. LTD.,ADM ASIA-PACIFIC TRADING PTE. LTD.,100
8,NAICS,424990,ADM ASIA-PACIFIC TRADING PTE. LTD.,ADM ASIA-PACIFIC TRADING PTE. LTD.,100
9,NAICS,423610,ARROW ELECTRONICS ASIA (S) PTE. LTD.,ARROW ELECTRONICS ASIA (S) PTE. LTD.,100


### CONCLUSION
> We have successfully matched all our 23 companies with accurate names. Although more can be done to make the code more robust to deal with other anomalies.